### In the next cell, the libraries and the file needed for the project are imported

In [46]:
import types
import pandas as pd
from botocore.client import Config
import ibm_boto3
import requests
import lxml.html as lh
import numpy as np
#!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim 
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
#!conda install -c conda-forge folium=0.5.0 --yes 
import folium
import json
from pandas.io.json import json_normalize

def __iter__(self): return 0

body = client_7a207381b1af4d15b126270709dfd5b8.get_object(Bucket='courseracapstoneproject-donotdelete-pr-x1dwa7gkvsd5xa',Key='Geospatial_Coordinates.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_lat_long = pd.read_csv(body)
df_lat_long.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


### Url is saved, as well as the table content

In [11]:
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
page = requests.get(url)
doc = lh.fromstring(page.content)
tr_elements = doc.xpath('//tr')

### Separate the first line of the table and save it in a list to be the table header

In [12]:
tr_elements = doc.xpath('//tr')
col=[]
i=0
for t in tr_elements[0]:
    i+=1
    name=t.text_content()
    print('%d:"%s"'%(i,name))
    col.append((name,[]))

1:"Postcode"
2:"Borough"
3:"Neighbourhood
"


### Add the following data to the list associating it with the data previously added

In [13]:
for j in range(1,len(tr_elements)):
    T=tr_elements[j]

    if len(T)!=3:
        break
    
    i=0
    
    for t in T.iterchildren():
        data=t.text_content() 

        col[i][1].append(data)

        i+=1

### Turn list into a dictionary and then into a dataframe

In [14]:
Dict={title:column for (title,column) in col}
df=pd.DataFrame(Dict)

### Remove rows in de dataframe where Borough is Not assigned

In [15]:
df = df[df.Borough != 'Not assigned']

### Rename df column 'Neighbourhood\n' to 'Neighborhood'

In [16]:
df = df.rename(columns={'Neighbourhood\n': 'Neighborhood'})
df.columns

Index(['Postcode', 'Borough', 'Neighborhood'], dtype='object')

### Remove /n from the Neighborhood column values

In [17]:
df['Neighborhood'] = df['Neighborhood'].str[:-1]

### Replace Neighbourhood column values that are equal to 'Not assigned' to the Borough column value of the same row

In [18]:
df.loc[df.Neighborhood == 'Not assigned', 'Neighborhood'] = df['Borough']

### Use concat to do an inner join with the dataframe from the file and the dataframe from the Url

In [20]:
result = pd.concat([df, df_lat_long], axis=1, join='inner')

### Drop the second Postcode column

In [26]:
result = result.drop(result.columns[3],axis=1)

In [28]:
result

,Postcode,Borough,Neighborhood,Latitude,Longitude
2,M3A,North York,Parkwoods,43.763573,-79.188711
3,M4A,North York,Victoria Village,43.770992,-79.216917
4,M5A,Downtown Toronto,Harbourfront,43.773136,-79.239476
5,M6A,North York,Lawrence Heights,43.744734,-79.239476
6,M6A,North York,Lawrence Manor,43.727929,-79.262029
7,M7A,Downtown Toronto,Queen's Park,43.711112,-79.284577
9,M9A,Queen's Park,Queen's Park,43.692657,-79.264848
10,M1B,Scarborough,Rouge,43.757410,-79.273304
11,M1B,Scarborough,Malvern,43.750072,-79.295849
13,M3B,North York,Don Mills North,43.781638,-79.304302


### Use geopy library to get the latitude and longitude values of Toronto.

In [34]:
address = 'Toronto, TO'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


### Create a map of Toronto

In [35]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(result['Latitude'], result['Longitude'], result['Borough'], result['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

### Get FourSquare API Credentials

In [36]:
CLIENT_ID = '###' 
CLIENT_SECRET = '###' 
VERSION = '20180605'

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: H4JL4VVIDPE3M5GBUVUNPZJEJ5S0DSMXQ0KRPY5VHZUDCWX3
CLIENT_SECRET:QOWHIAPRXGC0LA25GJ4OR2CC5BHMDZEO5RPUPEHZPWHJKEIV


### Focusing on the Downtown Toronto Borough

In [38]:
downtown_data = result[result['Borough'] == 'Downtown Toronto'].reset_index(drop=True)
downtown_data.head()

,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,Harbourfront,43.773136,-79.239476
1,M7A,Downtown Toronto,Queen's Park,43.711112,-79.284577
2,M5B,Downtown Toronto,Ryerson,43.836125,-79.205636
3,M5B,Downtown Toronto,Garden District,43.803762,-79.363452
4,M5C,Downtown Toronto,St. James Town,43.761631,-79.520999


In [39]:
address = 'Downtown Toronto, TO'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Downtown Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Downtown Toronto are 43.6541737, -79.38081164513409.


In [40]:
# create map of Manhattan using latitude and longitude values
map_downtown = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(downtown_data['Latitude'], downtown_data['Longitude'], downtown_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_downtown)  
    
map_downtown

### Getting nearby venues of a determined Neighbourhood

In [41]:
downtown_data.loc[0, 'Neighborhood']

'Harbourfront'

In [42]:
neighborhood_latitude = downtown_data.loc[0, 'Latitude'] 
neighborhood_longitude = downtown_data.loc[0, 'Longitude'] 

neighborhood_name = downtown_data.loc[0, 'Neighborhood'] 

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Harbourfront are 43.773136, -79.23947609999999.


In [44]:
LIMIT = 100
radius = 500
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=H4JL4VVIDPE3M5GBUVUNPZJEJ5S0DSMXQ0KRPY5VHZUDCWX3&client_secret=QOWHIAPRXGC0LA25GJ4OR2CC5BHMDZEO5RPUPEHZPWHJKEIV&v=20180605&ll=43.773136,-79.23947609999999&radius=500&limit=100'

In [47]:
req = requests.get(url).json()
req

{'meta': {'code': 200, 'requestId': '5e3a186529ce6a001b7a1675'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Woburn',
  'headerFullLocation': 'Woburn, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 8,
  'suggestedBounds': {'ne': {'lat': 43.7776360045, 'lng': -79.2332557734104},
   'sw': {'lat': 43.7686359955, 'lng': -79.24569642658957}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4b1711a6f964a520cbc123e3',
       'name': 'Federick Restaurant',
       'location': {'address': '1920 Ellesmere Rd',
        'crossStreet': 'at Bellamy Rd. N',
        'lat': 43.77469659057996,
        'lng': -79.24114242818267,
        'labeledLatLngs': [{'label': 'dis

In [48]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [50]:
venues = req['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Federick Restaurant,Hakka Restaurant,43.774697,-79.241142
1,Drupati's Roti & Doubles,Caribbean Restaurant,43.775222,-79.241678
2,Centennial Recreation Centre,Athletics & Sports,43.774593,-79.236500
3,Thai One On,Thai Restaurant,43.774468,-79.241268
4,TD Canada Trust,Bank,43.774952,-79.241343


In [51]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

8 venues were returned by Foursquare.


### Getting nearby venues of all the Neighbourhoods from Downtown Toronto

In [52]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [53]:
downtown_venues = getNearbyVenues(names=downtown_data['Neighborhood'],
                                   latitudes=downtown_data['Latitude'],
                                   longitudes=downtown_data['Longitude']
                                  )


Harbourfront
Queen's Park
Ryerson
Garden District
St. James Town
Berczy Park
Central Bay Street
Christie
Adelaide
King
Richmond
Harbourfront East
Toronto Islands
Union Station
Design Exchange
Toronto Dominion Centre


In [54]:
print(downtown_venues.shape)
downtown_venues.head()

(484, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Harbourfront,43.773136,-79.239476,Federick Restaurant,43.774697,-79.241142,Hakka Restaurant
1,Harbourfront,43.773136,-79.239476,Drupati's Roti & Doubles,43.775222,-79.241678,Caribbean Restaurant
2,Harbourfront,43.773136,-79.239476,Centennial Recreation Centre,43.774593,-79.236500,Athletics & Sports
3,Harbourfront,43.773136,-79.239476,Thai One On,43.774468,-79.241268,Thai Restaurant
4,Harbourfront,43.773136,-79.239476,TD Canada Trust,43.774952,-79.241343,Bank


### Counting Neighbourhood appearances

In [55]:
downtown_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Adelaide,87,87,87,87,87,87
Berczy Park,34,34,34,34,34,34
Central Bay Street,55,55,55,55,55,55
Christie,83,83,83,83,83,83
Design Exchange,3,3,3,3,3,3
Garden District,4,4,4,4,4,4
Harbourfront,8,8,8,8,8,8
Harbourfront East,24,24,24,24,24,24
King,16,16,16,16,16,16


### Counting unique venues

In [56]:
print('There are {} uniques categories.'.format(len(downtown_venues['Venue Category'].unique())))

There are 151 uniques categories.


### Adding venues as columns to dataframe

In [57]:
# one hot encoding
downtown_onehot = pd.get_dummies(downtown_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
downtown_onehot['Neighborhood'] = downtown_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [downtown_onehot.columns[-1]] + list(downtown_onehot.columns[:-1])
downtown_onehot = downtown_onehot[fixed_columns]

downtown_onehot.head()

,Neighborhood,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Art Gallery,...,Sushi Restaurant,Tailor Shop,Tea Room,Thai Restaurant,Toy / Game Store,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar,Women's Store,Yoga Studio
0,Harbourfront,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Harbourfront,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Harbourfront,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Harbourfront,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
4,Harbourfront,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [58]:
downtown_onehot.shape

(484, 152)

### Grouping by Neighborhood

In [59]:
downtown_grouped = downtown_onehot.groupby('Neighborhood').mean().reset_index()
downtown_grouped

,Neighborhood,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Art Gallery,...,Sushi Restaurant,Tailor Shop,Tea Room,Thai Restaurant,Toy / Game Store,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar,Women's Store,Yoga Studio
0,Adelaide,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.011494,0.011494,0.000000,0.045977,0.068966,0.011494,0.011494,0.000000
1,Berczy Park,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.000000,0.000000,...,0.058824,0.000000,0.000000,0.029412,0.029412,0.000000,0.000000,0.000000,0.000000,0.000000
2,Central Bay Street,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.000000,0.018182,...,0.000000,0.018182,0.018182,0.018182,0.000000,0.018182,0.000000,0.000000,0.000000,0.000000
3,Christie,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.012048,0.000000,0.000000,...,0.012048,0.000000,0.012048,0.012048,0.000000,0.012048,0.000000,0.012048,0.000000,0.012048
4,Design Exchange,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,Garden District,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,Harbourfront,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.125000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,Harbourfront East,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.041667,0.000000,...,0.000000,0.000000,0.000000,0.083333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,King,0.0625,0.0625,0.0625,0.125,0.1875,0.125,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,Queen's Park,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [91]:
downtown_grouped.shape

(15, 152)

### Top 5 venues by Neighborhoods 

In [100]:
num_top_venues = 5

for hood in downtown_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = downtown_grouped[downtown_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide----
                   venue  freq
0                   Café  0.07
1  Vietnamese Restaurant  0.07
2     Chinese Restaurant  0.06
3                    Bar  0.06
4            Coffee Shop  0.05


----Berczy Park----
            venue  freq
0     Pizza Place  0.09
1  Sandwich Place  0.09
2    Dessert Shop  0.09
3     Coffee Shop  0.06
4             Gym  0.06


----Central Bay Street----
                venue  freq
0         Coffee Shop  0.09
1        Cocktail Bar  0.05
2  Seafood Restaurant  0.04
3          Steakhouse  0.04
4      Farmers Market  0.04


----Christie----
                venue  freq
0         Coffee Shop  0.16
1      Sandwich Place  0.05
2  Italian Restaurant  0.05
3      Ice Cream Shop  0.05
4           Juice Bar  0.04


----Design Exchange----
                 venue  freq
0                 Park  0.67
1    Convenience Store  0.33
2              Airport  0.00
3               Lounge  0.00
4  Japanese Restaurant  0.00


----Garden District----
                     

In [101]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

### Top 10 venues by Neighborhoods 

In [102]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = downtown_grouped['Neighborhood']

for ind in np.arange(downtown_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(downtown_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Adelaide,Café,Vietnamese Restaurant,Chinese Restaurant,Bar,Coffee Shop,Vegetarian / Vegan Restaurant,Dumpling Restaurant,Mexican Restaurant,Burger Joint,Farmers Market
1,Berczy Park,Dessert Shop,Sandwich Place,Pizza Place,Gym,Coffee Shop,Café,Sushi Restaurant,Italian Restaurant,Seafood Restaurant,Pharmacy
2,Central Bay Street,Coffee Shop,Cocktail Bar,Beer Bar,Steakhouse,Bakery,Seafood Restaurant,Cheese Shop,Café,Farmers Market,Hotel
3,Christie,Coffee Shop,Sandwich Place,Ice Cream Shop,Italian Restaurant,Juice Bar,Burger Joint,Café,Japanese Restaurant,Salad Place,Bakery
4,Design Exchange,Park,Convenience Store,Yoga Studio,Cuban Restaurant,Discount Store,Diner,Dim Sum Restaurant,Dessert Shop,Department Store,Deli / Bodega


### Clustering Neighborhoods

In [103]:
kclusters = 5

downtown_grouped_clustering = downtown_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(downtown_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 1, 1, 1, 2, 3, 1, 1, 1, 1], dtype=int32)

### Adding Clustering labesls

In [104]:
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

downtown_merged = downtown_data

downtown_merged = downtown_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

downtown_merged.head() 

,Postcode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,Harbourfront,43.773136,-79.239476,1.0,Fried Chicken Joint,Gas Station,Caribbean Restaurant,Thai Restaurant,Hakka Restaurant,Bank,Bakery,Athletics & Sports,Department Store,Discount Store
1,M7A,Downtown Toronto,Queen's Park,43.711112,-79.284577,1.0,Bus Line,Bakery,Metro Station,Intersection,Bus Station,Park,Ice Cream Shop,Soccer Field,Yoga Studio,Deli / Bodega
2,M5B,Downtown Toronto,Ryerson,43.836125,-79.205636,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,M5B,Downtown Toronto,Garden District,43.803762,-79.363452,3.0,Pool,Golf Course,Mediterranean Restaurant,Dog Run,Yoga Studio,Diner,Dim Sum Restaurant,Dessert Shop,Department Store,Deli / Bodega
4,M5C,Downtown Toronto,St. James Town,43.761631,-79.520999,4.0,Grocery Store,Athletics & Sports,Discount Store,Gym / Fitness Center,Liquor Store,Diner,Dim Sum Restaurant,Dessert Shop,Department Store,Deli / Bodega


### Turning Cluster Labels column into integer type

In [111]:
downtown_merged = downtown_merged.dropna()
downtown_merged['Cluster Labels'] = downtown_merged['Cluster Labels'].astype(int)

### Creating map 

In [112]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(downtown_merged['Latitude'], downtown_merged['Longitude'], downtown_merged['Neighborhood'], downtown_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Examinging Clusters

In [113]:
downtown_merged.loc[downtown_merged['Cluster Labels'] == 0, downtown_merged.columns[[1] + list(range(5, downtown_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
15,Downtown Toronto,0,Pizza Place,Sandwich Place,Coffee Shop,Intersection,Chinese Restaurant,Middle Eastern Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store


In [114]:
downtown_merged.loc[downtown_merged['Cluster Labels'] == 1, downtown_merged.columns[[1] + list(range(5, downtown_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,1,Fried Chicken Joint,Gas Station,Caribbean Restaurant,Thai Restaurant,Hakka Restaurant,Bank,Bakery,Athletics & Sports,Department Store,Discount Store
1,Downtown Toronto,1,Bus Line,Bakery,Metro Station,Intersection,Bus Station,Park,Ice Cream Shop,Soccer Field,Yoga Studio,Deli / Bodega
5,Downtown Toronto,1,Dessert Shop,Sandwich Place,Pizza Place,Gym,Coffee Shop,Café,Sushi Restaurant,Italian Restaurant,Seafood Restaurant,Pharmacy
6,Downtown Toronto,1,Coffee Shop,Cocktail Bar,Beer Bar,Steakhouse,Bakery,Seafood Restaurant,Cheese Shop,Café,Farmers Market,Hotel
7,Downtown Toronto,1,Coffee Shop,Sandwich Place,Ice Cream Shop,Italian Restaurant,Juice Bar,Burger Joint,Café,Japanese Restaurant,Salad Place,Bakery
8,Downtown Toronto,1,Café,Vietnamese Restaurant,Chinese Restaurant,Bar,Coffee Shop,Vegetarian / Vegan Restaurant,Dumpling Restaurant,Mexican Restaurant,Burger Joint,Farmers Market
9,Downtown Toronto,1,Airport Service,Airport Lounge,Airport Terminal,Airport,Bar,Boutique,Boat or Ferry,Harbor / Marina,Sculpture Garden,Coffee Shop
10,Downtown Toronto,1,Coffee Shop,Café,Beer Bar,Cocktail Bar,Hotel,Restaurant,Seafood Restaurant,Italian Restaurant,Japanese Restaurant,Art Gallery
11,Downtown Toronto,1,Mexican Restaurant,Thai Restaurant,Bar,Café,Furniture / Home Store,Cajun / Creole Restaurant,Bookstore,Park,Italian Restaurant,Music Venue
12,Downtown Toronto,1,Breakfast Spot,Gift Shop,Dog Run,Dessert Shop,Bookstore,Italian Restaurant,Bar,Eastern European Restaurant,Cuban Restaurant,Coffee Shop


In [115]:
downtown_merged.loc[downtown_merged['Cluster Labels'] == 2, downtown_merged.columns[[1] + list(range(5, downtown_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
14,Downtown Toronto,2,Park,Convenience Store,Yoga Studio,Cuban Restaurant,Discount Store,Diner,Dim Sum Restaurant,Dessert Shop,Department Store,Deli / Bodega


In [116]:
downtown_merged.loc[downtown_merged['Cluster Labels'] == 3, downtown_merged.columns[[1] + list(range(5, downtown_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,Downtown Toronto,3,Pool,Golf Course,Mediterranean Restaurant,Dog Run,Yoga Studio,Diner,Dim Sum Restaurant,Dessert Shop,Department Store,Deli / Bodega


In [117]:
downtown_merged.loc[downtown_merged['Cluster Labels'] == 4, downtown_merged.columns[[1] + list(range(5, downtown_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,Downtown Toronto,4,Grocery Store,Athletics & Sports,Discount Store,Gym / Fitness Center,Liquor Store,Diner,Dim Sum Restaurant,Dessert Shop,Department Store,Deli / Bodega
